# Elliptic curve cryptography 

## Tutorial 1

In [ ]:
extern crate kn0syseccrs as ecc;
extern crate num;
extern crate hex;

> Treat an elliptic curve group of points like you do vectors:
  you can add/subtract points `(G + H, G - H)` 
  and you can do scalar multiplication with it `(x * G or xG for short)`.
 ...and the analogy stops there.

> However, we can have a vector/array of scalars and of points (part 2).

> Difference #1: The scalars. In vector calculus, scalars are real
  numbers. On the other hand, our scalar is integers modulo a large
  prime number l. in other words, our scalars are only from 0 to (l - 1)
 (the remainders when any integer is divided by l).
  Here's the l:
>
> Reference: https://datatracker.ietf.org/doc/html/rfc8032#section-5.1

In [ ]:
let big_int = num::BigInt::from_bytes_le(num::bigint::Sign::Plus, &hex::decode(ecc::L).unwrap_or_default());
println!("L: {:?}", big_int);
println!("invalid: {:?}", ecc::Scalar::new(big_int));

## Scalar Addition

In [ ]:
let s1 = ecc::Scalar::new(num::BigInt::from(1)).unwrap();
let s1h = ecc::Scalar::new(num::BigInt::from(1)).unwrap().get_hex();
let s2 = ecc::Scalar::new(num::BigInt::from(2)).unwrap();
let s2h = ecc::Scalar::new(num::BigInt::from(2)).unwrap().get_hex();
let result = s1 + s2;
println!("{:?} + {:?} = {:?}", s1h, s2h, result.unwrap().get_hex());

## Scalar Subtraction

In [ ]:
let s1 = ecc::Scalar::new(num::BigInt::from(1)).unwrap();
let s1h = ecc::Scalar::new(num::BigInt::from(1)).unwrap().get_hex();
let s2 = ecc::Scalar::new(num::BigInt::from(2)).unwrap();
let s2h = ecc::Scalar::new(num::BigInt::from(2)).unwrap().get_hex();
let result = s1 - s2;
println!("{:?} - {:?} = {:?}", s1h, s2h, result.unwrap().get_hex());

## Scalar Multiplication

In [ ]:
let s1 = ecc::Scalar::new(num::BigInt::from(2)).unwrap();
let s1h = ecc::Scalar::new(num::BigInt::from(2)).unwrap().get_hex();
let s2 = ecc::Scalar::new(num::BigInt::from(3)).unwrap();
let s2h = ecc::Scalar::new(num::BigInt::from(3)).unwrap().get_hex();
let result = s1 * s2;
println!("{:?} * {:?} = {:?}", s1h, s2h, result.unwrap().get_hex());

## Scalar Division

> We have something like "division", but we do not use slash.
> Instead, inversion (analogous to "reciprocal") is performed on the supposed
> divisor, then perform multiplication.


In [ ]:
let s1 = ecc::Scalar::new(num::BigInt::from(1)).unwrap();
let s2 = ecc::Scalar::new(num::BigInt::from(2)).unwrap();
let result = s1.divide(s2);
println!("x * 1/x = {:?}", result.unwrap().get_hex());

## Scalar Exponentiation

In [ ]:
// This library uses num::BigInt which only accepts usize for exponents
let s1 = ecc::Scalar::new(num::BigInt::from(2)).unwrap();
let s1h = ecc::Scalar::new(num::BigInt::from(2)).unwrap().get_hex();
let s2 = ecc::Scalar::new(num::BigInt::from(3)).unwrap();
let result = s1.pow(3);
println!("{:?} * {:?} = {:?}", s1h, 3, result.unwrap().get_hex());

## Random Scalar

In [ ]:
let r = ecc::Scalar::random().unwrap();
println!("{:?}", r.get_hex());

> Differences #2: The elliptic curve points. These are actually points (x,y)
  but the x and y are integers modulo another large (not necessarily prime)
  number q. We usually do not initialize points like we initialize scalar. Instead, we use
  the base generator.
>
> Reference: [crytpography stack exchange](https://crypto.stackexchange.com/questions/27392/base-point-in-ed25519)

In [ ]:
let g = ecc::Point::base_generator().unwrap();
println!("G: {:?}", g.get_hex());

## Point Addition and Subtraction

In [ ]:
let g1 = ecc::Point::base_generator().unwrap();
let g2 = ecc::Point::base_generator().unwrap();
let g3 = ecc::Point::base_generator().unwrap();
let g4 = ecc::Point::base_generator().unwrap();
let sum = g1 + g2;
let diff = g3 - g4;
println!("G + G = {:?}", sum.unwrap().get_hex());
println!("G - G = {:?}", diff.unwrap().get_hex());

## Zero Point "Z"

In [ ]:
let z = ecc::Point::zero().unwrap();
println!("Z = {:?}", z.get_hex());

> "Are G - G and Z the same?"

In [ ]:
for i in 1..15 {
    let g = ecc::Point::base_generator().unwrap();
    let s = ecc::Scalar::new(num::BigInt::from(i)).unwrap();
    let xG = g * s;
    println!("{} * G: {:?}", i, xG.unwrap().get_hex());
}

> Those last points look "random". This IS a big reason why we use elliptic curves in cryptography:
   
> If I give you a random point P, it is assumed to be 
  impossible to find the x such that `P = xG`. The problem of finding x is called "Discrete
> Logarithm Problem" (DLP) and the impossibility assumption is called Discrete Logarithm (DL)
> assumption."

### exercise: what is (-1)G + G? 

In [ ]:
// code here

### exercise: is Z == Z + random_point()?

In [ ]:
// code here

### cryptographic hash functions

In [ ]:
// pass a vector of string slices
let s1 = ecc::Scalar::new(num::BigInt::from(12)).unwrap();
let s1_str = String::from(s1.get_hex());
let v = vec!["tutorial", &s1_str];
let s2 = ecc::hash_to_scalar(v).unwrap();
println!("{:?}", s2.get_hex());

### Exercise: the Diffie-Hellman (DH) key exchange
>  Implement DH key exchange (just use variables):
   Alice and Bob wants to share a secret scalar only they would know.
   Using the generator G and hashing, how would they do it?
   show that after the key exchange, Alice and Bob has a shared secret.

In [ ]:
// code here

### Exercise: implement Elgamal point encryption scheme.

here's the scenario:
  * Alice must send the point Y to Bob securely. Bob generates a random keypair `(x, xG)`.
  * x is the private key, and `P = xG` is the public key to be shared to Alice. Alice encrypts
  * Y using P, and sends the cipher to Bob. Bob then decrypts the cipher using x.

just like in DH key exchange, just use variables.
  * encryption: given a point Y and point P, it must output a pair `(rG, Y + rP)`where r is a random scalar.
  * decryption: given a cipher pair (C1, C2) and a scalar x, output `Y = C2 - x * C1`.

then demonstrate the homomorphicity of Elgamal encryption scheme. using
  * two plaintexts `69000 * H and 420 * H`, encrypt both separately, then pairwise add the two ciphers,
  * then decrypt the "sum" cipher. what is the decrypted plaintext? 

In [ ]:
// code here